<a href="https://colab.research.google.com/github/NitinMohan0802/BigMart-Sales-prediction-/blob/main/BigMart_Sales_Pridictionby_NITIN_MOHAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NITIN MOHAN NOTEBOOK**

In [1]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/archive (2).zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


# **IMPORTING LIBRARIES**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

# **DATA COLLECTION & ANALYSIS**

In [3]:
# Loading the dataset from csv file to a pandas DataFrame
big_mart_data=pd.read_csv('/content/Train.csv')
big_mart_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
# Nimber of data points & Number of Features
big_mart_data.shape

(8523, 12)

In [5]:
# Getting information about the dataset
big_mart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


**Categorical Feature:**
- Item_Identifier
- Item_Fat_Content
- Item_Type
-Outlet_Identifier
-Outlet_Size
-Outlet_Location_Type
- Outlet_Type

**Missing Values**

In [6]:
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

**Handling Missing Values**
- Mean -> Average Values
- Mode -> Most repeated values

In case of `Numerical Values` we handle missing values through through imputation technique (`putting mean`)

In case of `Categorical Values` we handle missing values through through imputation technique (`putting Mode`) because we can not find the mean of Categorical variable.

In [7]:
# Mean value of "item Weight" column
big_mart_data['Item_Weight'].mean()

12.857645184135976

In [9]:
# Filling the Missing values in "Item Weight" column with "Mean" Values
big_mart_data['Item_Weight'].fillna(big_mart_data['Item_Weight'].mean(), inplace= True )
# inplace = True is used to make change in original dataset not only in here of that columns; If not mentioned give you relaced value only in this cell not change in our orginal d ataset

In [10]:
# We can observe that there is no missing value in Item Weight columns
big_mart_data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [12]:
# mode of "Outlet_Size" column
big_mart_data['Outlet_Size'].mode()
# From here we got the mode of 'Medium' and assigning every missing values to Medium is not a good practice what if it is a small or large

0    Medium
Name: Outlet_Size, dtype: object

To find the exact and accurate value to replace in outlet size we can perform feature extraction with outlet type means that we can compare our outlet size with outlet type and based on that we can impute the mode in outlet size.

In [13]:
# filling the missing values in "Outlet_Size" column with Mode
mode_of_Outlet_size = big_mart_data.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=(lambda x: x.mode()[0]))

As our two columns are correlated we from where we got the right mode that we have to insert

Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2 Supermarket Type3

Outlet_Size         Small             Small            Medium            Medium

from the above it want to say that if it is grocery or Supermarket Type1 then the outlet size is small and if it is Supermarket Type2 Supermarket Type3 it will be Medium

In [14]:
print(mode_of_Outlet_size)

Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2 Supermarket Type3
Outlet_Size         Small             Small            Medium            Medium


In [15]:
miss_values = big_mart_data['Outlet_Size'].isnull()

In [16]:
print(miss_values) # if values missing then it will be true

0       False
1       False
2       False
3        True
4       False
        ...  
8518    False
8519     True
8520    False
8521    False
8522    False
Name: Outlet_Size, Length: 8523, dtype: bool


In [17]:
big_mart_data.loc[miss_values, 'Outlet_Size'] = big_mart_data.loc[miss_values,'Outlet_Type'].apply(lambda x: mode_of_Outlet_size[x])

In [18]:
# checking for missing values
big_mart_data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

**Data Analysis**